In [1]:


import tensorflow as tf
from tensorflow import keras
from keras import layers

(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
x_val, y_val = x_train[-5000:], y_train[-5000:]
x_train, y_train = x_train[:-5000], y_train[:-5000]

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(64)
val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(64)


In [2]:

class MyCNN(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = layers.Conv2D(32, 3, activation='relu')
        self.pool = layers.MaxPooling2D()
        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(128, activation='relu')
        self.out = layers.Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.dense1(x)
        return self.out(x)

model = MyCNN()

loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.Adam()

train_acc = keras.metrics.SparseCategoricalAccuracy()
val_acc = keras.metrics.SparseCategoricalAccuracy()


In [3]:

for epoch in range(5):
    print(f"\nEpoch {epoch+1}/5")
    for x_batch, y_batch in train_ds:
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc.update_state(y_batch, logits)

    print(f"Train Accuracy: {train_acc.result().numpy():.4f}")
    train_acc.reset_state()

    for x_batch, y_batch in val_ds:
        preds = model(x_batch, training=False)
        val_acc.update_state(y_batch, preds)
    print(f"Val Accuracy: {val_acc.result().numpy():.4f}")
    val_acc.reset_state()



Epoch 1/5
Train Accuracy: 0.8443
Val Accuracy: 0.8750

Epoch 2/5
Train Accuracy: 0.8945
Val Accuracy: 0.9010

Epoch 3/5
Train Accuracy: 0.9083
Val Accuracy: 0.9108

Epoch 4/5
Train Accuracy: 0.9204
Val Accuracy: 0.9108

Epoch 5/5
Train Accuracy: 0.9300
Val Accuracy: 0.9062


In [4]:

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(64)
test_acc = keras.metrics.SparseCategoricalAccuracy()
for x_batch, y_batch in test_ds:
    preds = model(x_batch, training=False)
    test_acc.update_state(y_batch, preds)
print(f"\nTest Accuracy: {test_acc.result().numpy():.4f}")



Test Accuracy: 0.8989
